# Process USGS streamflow

### Prepare Workspace

In [9]:
# Import system libraries
import os
import sys

# Import data manipulation librariaes
import datetime
import pandas as pd
import numpy as np

# Import API libraries
import requests
import csv

# Set working directory
os.chdir('/Users/jessicarapson/Documents/GitHub/water-supply-forecast')

### Load Data from Folders

In [49]:
metadata = pd.read_csv("./assets/data/metadata.csv")

# Create empty list
usbr_all = []

# Loop through years
for year in range(1985, 2025):
    print(f"#######Retreiving data for {year}#######")

    # Loop through forecasting sites
    for site in metadata['site_id']:
        file_path = f"./assets/data/usgs_streamflow/FY{year}/{site}.csv"

        #  # Check if the file exists for the station
        if not os.path.exists(file_path):
            print(f"File not found for site {site}. Skipping...")
            continue

        # Read in daily flow data for the selected year
        flow_data = pd.read_csv(file_path)
        flow_data['site_id'] = site

        # Select data of interest
        flow_data = flow_data[['site_id', 'datetime', '00060_Mean']]

        # Append to list
        usbr_all.append(flow_data)

# Combine data
result = pd.concat(usbr_all, axis=0, ignore_index=True)

# Perform additional cleaning
result['datetime'] = pd.to_datetime(result['datetime']).dt.strftime('%Y-%m-%d')
result['00060_Mean'] = np.where(result['00060_Mean'] == -999999, np.nan, result['00060_Mean'])

# Filter to years of interest
result_week = result.copy()
result_week['date'] = pd.to_datetime(result['datetime']).dt.date
result_week = result_week[pd.to_datetime(
    result_week['date']) >= pd.Timestamp("1984-01-01")]

# Define start and end dates
start_date = datetime.date(1984, 1, 1)
end_date = datetime.date(2024, 1, 1)

# Initialize an empty list to store weeks
week_list = []

# Generate weeks between start_date and end_date
current_date = start_date
while current_date < end_date:
    for day in [1, 8, 15, 22]:
        week = current_date + datetime.timedelta(days=(day - current_date.weekday() - 1))
        if week < end_date:
            week_list.append(week.strftime('%Y-%m-%d'))
    current_date += datetime.timedelta(days=7)
    
# Function to round down the day to the nearest value less than or equal to the day
def round_day_down(date):
    day = date.day
    nearest_values = [1, 8, 15, 22]

    # Find the nearest value less than or equal to the day
    rounded_day = max(filter(lambda x: x <= day, nearest_values))
    return date.replace(day=rounded_day)

 # Create a new column 'Rounded_Day_Column' based on 'Date_Column'
result_week['week_start_date'] = result_week['date'].apply(round_day_down)
result_week = result_week.drop(['date', 'datetime'], axis=1)

# Aggregate by week
result_week = result_week.groupby(
    ['site_id','week_start_date']).mean().reset_index()

# Export data
result_week.to_csv('./assets/data/usgs_streamflow/usgs_streamflow.csv', index=False)

File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not

File not found for site pueblo_reservoir_inflow. Skipping...
File not found for site sweetwater_r_nr_alcova. Skipping...
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site boysen_reservoir_inflow. Skipping...
File not found for site pueblo_reservoir_inflow. Skipping...
File not found for site sweetwater_r_nr_alcova. Skipping...
File not found for site boise_r_nr_boise. Skipping...
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site boysen_reservoir_inflow. Skipping...
File not found for site pueblo_reservoir_inflow. Skipping...
File not found for site s

In [23]:
# Test for largest relationship to target variable